In [2]:
def get_XYR(Ra,Dec,cent_R,cent_D):
    Ac = np.radians(cent_R) #center of RA
    Dc = np.radians(cent_D)
    X = []
    Y = []
    R = []
    for i in range(len(Ra)):
        a = np.radians(Ra[i])
        d = np.radians(Dec[i])
        x = -180/np.pi*(np.cos(d)*np.sin(a-Ac))
        y = 180/np.pi*(np.sin(d)*np.cos(Dc)-np.cos(d)*np.sin(Dc)*np.cos(a-Ac))
        R_cut = x**2 + y**2
        X.append(x)
        Y.append(y)
        R.append(R_cut**(0.5))
    return(X,Y,R)


In [3]:

#CMD selection
def convert_CMD_for_dwarf(Dist):
    Dist_modula = 5*np.log10(Dist*1000)-5-19.40407#-0.3

    box_g_rp_0 = [[.3, 21.5+.3], [.3, 21+.3],[.3, 20.7+.3],[-0.1, 20.7+.3],[-0.3, 20.7+.3],[-0.3, 20.1],[.3, 19.3],[.45, 18.8],[.5, 18],
                  [.6, 17],[.74, 16], [1.06+0.03, 16],  [.95, 17],[.84, 18],[.77+0.01, 19],[.75+0.04, 19.5], [.73+0.06, 20],
                  [.71+0.1, 20.5],
                  [.68+0.15, 21], [.87, 21.5]]
    box_g_rp_0.append(box_g_rp_0[0])
    box_g_rp_0 = np.array(box_g_rp_0)
    for i in range(len(box_g_rp_0)):
    #box_g_rp_0.T[1][i] +=(.741)
        box_g_rp_0.T[1][i] += (Dist_modula)
    return(box_g_rp_0)




def Deg_XY_check(a,d,Ac,Dc,R_min,R_max):
    UL = R_max**2
    LL = R_min**2
    AcR = np.radians(Ac)
    DcR = np.radians(Dc)
    ar = np.radians(a)#in radian
    dr = np.radians(d)
    x = -180/np.pi*(np.cos(dr)*np.sin(ar-AcR))
    y = 180/np.pi*(np.sin(dr)*np.cos(DcR)-np.cos(dr)*np.sin(DcR)*np.cos(ar-AcR))
    R_cut = x**2 + y**2
    return(LL <= R_cut <= UL, x,y)


def Cuts_for_dwarf(Deg_7,dwarf_CMD,Cent_RA,Cent_DEC,LL,UL):
    coord = zip(dwarf_CMD[:,0],dwarf_CMD[:,1])
    poly = Polygon(coord)
    X = []
    Y = []
    RA = []
    DEC = []
    PLX = []
    PLX_E = []
    ID = []
    PMRA = []
    PMDEC = []
    EPMRA = []
    EPMDEC = []
    RV = []
    G = []
    GGR = []
    G_obs = []
    GGR_obs = []
    for i in range(len(Deg_7)):
        ra_all = Deg_7[i]['ra'].data
        dec_all = Deg_7[i]['dec'].data
        plx_all = Deg_7[i]['plx'].data
        plx_e_all = Deg_7[i]['plx_e'].data
        pmra_all = Deg_7[i]['pmra'].data
        pmdec_all = Deg_7[i]['pmdec'].data
        epmra_all = Deg_7[i]['epmra'].data
        epmdec_all = Deg_7[i]['epmdec'].data
        rv_all  = Deg_7[i]['vr'].data
        id_all = Deg_7[i]['ParticleID'].data
        g = Deg_7[i]['g'].data
        gr = Deg_7[i]['gr'].data
        ggr = g-gr
        g_obs = Deg_7[i]['g_obs'].data
        gr_obs = Deg_7[i]['gr_obs'].data
        ggr_obs = g_obs-gr_obs
        for j in range(len(ra_all)):
            pt = Point(ggr[j],g[j])
            Check_XY = Deg_XY_check(ra_all[j],dec_all[j],Cent_RA,Cent_DEC,LL,UL)
            if Check_XY[0] and pt.within(poly):
                X.append(Check_XY[1])
                Y.append(Check_XY[2])
                RA.append(ra_all[j])
                DEC.append(dec_all[j])
                PLX.append(plx_all[j])
                PLX_E.append(plx_e_all[j])
                PMRA.append(pmra_all[j])
                PMDEC.append(pmdec_all[j])
                EPMRA.append(epmra_all[j])
                EPMDEC.append(epmdec_all[j])
                ID.append(id_all[j])
                RV.append(rv_all[j])
                G.append(g[j])
                GGR.append(ggr[j])
                G_obs.append(g_obs[j])
                GGR_obs.append(ggr_obs[j])
    res_table = Table([ID,X,Y,RA,DEC,PLX,PLX_E,PMRA,PMDEC,EPMRA,EPMDEC,RV,G,GGR,G_obs,GGR_obs],names=('id','x','y','ra','dec','plx','eplx','pmra','pmdec','epmra','epmdec','rv','g','ggr','g_obs','ggr_obs'))
    return(res_table)


In [4]:
def Sigma_Error_Draco(Dra):
    Sigma_EPMRA = []
    Sigma_EPMDEC = []
    Sigma_EPLX = []
    E_PMRA = Dra['pmra_error']
    E_PMDEC = Dra['pmdec_error']
    E_PLX = Dra['plxe']
    m_ra = np.mean(E_PMRA)
    m_dec = np.mean(E_PMDEC)
    m_plx = np.mean(E_PLX)
    for i in range(len(E_PMRA)):
        Sigma_EPMRA.append(E_PMRA[i]-m_ra)
        Sigma_EPMDEC.append(E_PMDEC[i]-m_dec)
        Sigma_EPLX.append(E_PLX[i]-m_plx)
    return(Sigma_EPMRA,Sigma_EPMDEC,Sigma_EPLX)
    

def Calculate_Error_EDR3(d_table,Dra):
    EPMRA = []
    EPMDEC = []
    EPLX = []
    G = d_table['g']
    for i in range(len(G)):
        g = G[i]
        z = max(10**(0.4*(13 - 15)),10**(0.4*(g - 15)))
        Omega = (40 + 80*z + 30*z**2)**(0.5)/1000
        epmra = 1.03*Omega
        epmdec = 0.89*Omega
        
        Dra_1 = Dra[(Dra['g']<=g+0.1)]
        Dra_2 = Dra_1[(Dra_1['g']>=g-0.1)]
        Sigma = Sigma_Error_Draco(Dra_2)
        
        try: 
            random_index = random.randint(0,len(Sigma[0])-1)
            rdm_ra = Sigma[0][random_index]
            rdm_dec = Sigma[1][random_index]
            rdm_plx = Sigma[2][random_index]
        except:
            random_index = 0
            rdm_ra = 0
            rdm_dec = 0
            rdm_plx = 0
        EPMRA.append(epmra+rdm_ra)
        EPMDEC.append(epmdec+rdm_dec)
        EPLX.append(Omega+rdm_plx)
    res_table = Table([d_table['id'],d_table['x'],d_table['y'],d_table['ra'],d_table['dec'],d_table['plx'],EPLX,d_table['pmra'],d_table['pmdec'],EPMRA,EPMDEC,d_table['rv'],d_table['g'],d_table['ggr'],d_table['g_obs'],d_table['ggr_obs']],names=('ParticleID','x','y','ra','dec','plx','eplx','pmra','pmdec','epmra','epmdec','rv','g','ggr','g_obs','ggr_obs'))
    return(res_table)


def Calculate_Value_EDR3(d_table):
    PMRA_res = []
    PMDEC_res = []
    PLX_res = []
    for i in range(len(d_table['eplx'])):
        eplx = d_table['eplx'][i]
        epmra = d_table['epmra'][i]
        epmdec = d_table['epmdec'][i]
        plx = np.random.normal(loc=d_table['plx'][i], scale=eplx)
        pmra = np.random.normal(loc=d_table['pmra'][i], scale=epmra)
        pmdec = np.random.normal(loc=d_table['pmdec'][i], scale=epmdec)
        PMRA_res.append(pmra)
        PMDEC_res.append(pmdec)
        PLX_res.append(plx)
    Target_zeros = np.zeros(len(d_table), dtype=float)
    res_table = Table([d_table['ParticleID'],d_table['x'],d_table['y'],d_table['ra'],d_table['dec'],PLX_res,d_table['eplx'],PMRA_res,PMDEC_res,d_table['epmra'],d_table['epmdec'],d_table['rv'],d_table['g'],d_table['ggr'],d_table['g_obs'],d_table['ggr_obs'],Target_zeros,Target_zeros,Target_zeros,Target_zeros],names=('ParticleID','x','y','ra','dec','plx','eplx','pmra','pmdec','epmra','epmdec','rv','g','ggr','g_obs','ggr_obs','prob_pmra','prob_pmdec','pmra_pmdec_corr','prob_pmra_pmdec'))
    return(res_table)

In [5]:
# Andrew's code on calulating the MW backgroud 
def MW_model_p(dwarf_selection_targets,dwarf_selection_far):
    for i in range(len(dwarf_selection_targets)):
        prob_1 = np.sum(np.exp(-.5 * (dwarf_selection_far['pmra']-dwarf_selection_targets['pmra'][i])**2/ \
                                   (dwarf_selection_targets['epmra'][i]**2 + dwarf_selection_far['epmra']**2))/ \
                            np.sqrt(2.*np.pi * (dwarf_selection_targets['epmra'][i]**2 + dwarf_selection_far['epmra']**2)))
        prob_2 = np.sum(np.exp(-.5 * (dwarf_selection_far['pmdec']-dwarf_selection_targets['pmdec'][i])**2/ \
                                   (dwarf_selection_targets['epmdec'][i]**2 + dwarf_selection_far['epmdec']**2))/ \
                            np.sqrt(2.*np.pi * (dwarf_selection_targets['epmdec'][i]**2 + dwarf_selection_far['epmdec']**2)))
        dwarf_selection_targets['prob_pmra'][i] = prob_1/len(dwarf_selection_far)
        dwarf_selection_targets['prob_pmdec'][i] = prob_2/len(dwarf_selection_far)
        #prob_both = np.nansum(np.exp(-.5 * ((dwarf_selection_targets['epmdec'][i]**2 + dwarf_selection_far['epmdec']**2)*(dwarf_selection_far['pmra']-dwarf_selection_targets['pmra'][i])**2 \
        #                           + (dwarf_selection_targets['pmra_error'][i]**2 + dwarf_selection_far['pmra_error']**2)*(dwarf_selection_far['pmdec']-dwarf_selection_targets['pmdec'][i])**2 \
        #    -2.*(dwarf_selection_far['pmdec']-dwarf_selection_targets['pmdec'][i])*(dwarf_selection_far['pmra']-dwarf_selection_targets['pmra'][i]) \
        #                                     *dwarf_selection_far['pmra_pmdec_corr']*dwarf_selection_targets['pmra_pmdec_corr'][i])/ \
        #((dwarf_selection_targets['epmra'][i]**2 + dwarf_selection_far['epmra']**2)*(dwarf_selection_targets['epmdec'][i]**2 + dwarf_selection_far['epmdec']**2) \
        #    -(dwarf_selection_far['pmra_pmdec_corr'] + dwarf_selection_targets['pmra_pmdec_corr'][i])**2) )/ \
        #                    np.sqrt(4.*np.pi**2 * \
        #     ((dwarf_selection_targets['epmra'][i]**2 + dwarf_selection_far['epmra']**2)*(dwarf_selection_targets['epmdec'][i]**2 + dwarf_selection_far['epmdec']**2) \
        #    -(dwarf_selection_far['pmra_pmdec_corr'] + dwarf_selection_targets['pmra_pmdec_corr'][i])**2)
        #                       ))
        dwarf_selection_targets['prob_pmra_pmdec'][i] = dwarf_selection_targets['prob_pmra'][i]*dwarf_selection_targets['prob_pmdec'][i]
    return(dwarf_selection_targets['prob_pmra_pmdec'])

def Plot_MW_model(dwarf_selection_targets,dwarf_selection_far):   
    pmra_value = np.arange(np.min(dwarf_selection_far['pmra']),np.max(dwarf_selection_far['pmra']), .025)
    pmdec_value = np.arange(np.min(dwarf_selection_far['pmdec']),np.max(dwarf_selection_far['pmdec']), .025)
    pmra_kde_mw = []
    pmdec_kde_mw = []
    for i in pmra_value:
        temp = np.sum(np.exp(-.5 * (dwarf_selection_far['pmra']-i)**2/dwarf_selection_far['epmra']**2)/np.sqrt(2.*np.pi)\
            /dwarf_selection_far['epmra'])
        pmra_kde_mw.append(temp)
    pmra_kde_mw =np.array(pmra_kde_mw)
    for i in pmdec_value:
        temp = np.sum(np.exp(-.5 * (dwarf_selection_far['pmdec']-i)**2/dwarf_selection_far['epmdec']**2)/np.sqrt(2.*np.pi)\
            /dwarf_selection_far['epmdec'])
        pmdec_kde_mw.append(temp)
    pmdec_kde_mw =np.array(pmdec_kde_mw)
    
    pmra_kde_target = []
    pmdec_kde_target = []
    for i in pmra_value:
        temp = np.sum(np.exp(-.5 * (dwarf_selection_targets['pmra']-i)**2/dwarf_selection_targets['epmra']**2)/np.sqrt(2.*np.pi)\
            /dwarf_selection_targets['epmra'])

        pmra_kde_target.append(temp)
    pmra_kde_target =np.array(pmra_kde_target)
    for i in pmdec_value:
        temp = np.sum(np.exp(-.5 * (dwarf_selection_targets['pmdec']-i)**2/dwarf_selection_targets['epmdec']**2)/np.sqrt(2.*np.pi)\
            /dwarf_selection_targets['epmdec'])
        pmdec_kde_target.append(temp)
    pmdec_kde_target =np.array(pmdec_kde_target)
    return(pmra_value, pmra_kde_mw/np.max(pmra_kde_mw),pmra_kde_target/np.max(pmra_kde_target),pmdec_value, pmdec_kde_mw/np.max(pmdec_kde_mw), pmdec_kde_target/np.max(pmdec_kde_target))
    



In [6]:
def MW_model_p_tot(dwarf_selection_targets,dwarf_selection_far):
    for i in range(len(dwarf_selection_targets)):
        prob_1 = np.sum(np.exp(-.5 * (dwarf_selection_far['pmra']-dwarf_selection_targets['pmra'][i])**2/ \
                                   (dwarf_selection_targets['epmra'][i]**2 + dwarf_selection_far['epmra']**2))/ \
                            np.sqrt(2.*np.pi * (dwarf_selection_targets['epmra'][i]**2 + dwarf_selection_far['epmra']**2)))
        prob_2 = np.sum(np.exp(-.5 * (dwarf_selection_far['pmdec']-dwarf_selection_targets['pmdec'][i])**2/ \
                                   (dwarf_selection_targets['epmdec'][i]**2 + dwarf_selection_far['epmdec']**2))/ \
                            np.sqrt(2.*np.pi * (dwarf_selection_targets['epmdec'][i]**2 + dwarf_selection_far['epmdec']**2)))
        dwarf_selection_targets['prob_pmra'][i] = prob_1/len(dwarf_selection_far)
        dwarf_selection_targets['prob_pmdec'][i] = prob_2/len(dwarf_selection_far)
        prob_both = np.nansum(np.exp(-.5 * ((dwarf_selection_targets['epmdec'][i]**2 + dwarf_selection_far['epmdec']**2)*(dwarf_selection_far['pmra']-dwarf_selection_targets['pmra'][i])**2 \
                                   + (dwarf_selection_targets['epmra'][i]**2 + dwarf_selection_far['epmra']**2)*(dwarf_selection_far['pmdec']-dwarf_selection_targets['pmdec'][i])**2 \
            -2.*(dwarf_selection_far['pmdec']-dwarf_selection_targets['pmdec'][i])*(dwarf_selection_far['pmra']-dwarf_selection_targets['pmra'][i]) \
                                             *dwarf_selection_far['pmra_pmdec_corr']*dwarf_selection_targets['pmra_pmdec_corr'][i])/ \
        ((dwarf_selection_targets['epmra'][i]**2 + dwarf_selection_far['epmra']**2)*(dwarf_selection_targets['epmdec'][i]**2 + dwarf_selection_far['epmdec']**2) \
            -(dwarf_selection_far['pmra_pmdec_corr'] + dwarf_selection_targets['pmra_pmdec_corr'][i])**2) )/ \
                            np.sqrt(4.*np.pi**2 * \
             ((dwarf_selection_targets['epmra'][i]**2 + dwarf_selection_far['epmra']**2)*(dwarf_selection_targets['epmdec'][i]**2 + dwarf_selection_far['epmdec']**2) \
            -(dwarf_selection_far['pmra_pmdec_corr'] + dwarf_selection_targets['pmra_pmdec_corr'][i])**2)
                               ))
        #dwarf_selection_targets['prob_pmra_pmdec'][i] = dwarf_selection_targets['prob_pmra'][i]*dwarf_selection_targets['prob_pmdec'][i]
        dwarf_selection_targets['prob_pmra_pmdec'][i] = prob_both/len(dwarf_selection_far)
    return(dwarf_selection_targets['prob_pmra_pmdec'])



In [7]:
def calc_cov_wcs(data, disp):
    cov = np.zeros((len(data), 2, 2))
    cov[:,0,0] = data['epmra']**2 + disp**2
    cov[:,0,1] = data['epmra'] * data['epmdec'] * data['pmra_pmdec_corr']
    cov[:,1,0] = data['epmra'] * data['epmdec'] * data['pmra_pmdec_corr']
    cov[:,1,1] = data['epmdec']**2 + disp**2
    
    cov2 = np.zeros((len(data), 2, 2))
    cov2[:,0,0] = data['epmra']**2
    cov2[:,0,1] = data['epmra'] * data['epmdec'] * data['pmra_pmdec_corr']
    cov2[:,1,0] = data['epmra'] * data['epmdec'] * data['pmra_pmdec_corr']
    cov2[:,1,1] = data['epmdec']**2 
        
    return cov, cov2


#Version of the spatial model that includes the ellipticity as a free parameter
def spat_model_ellip(cx, cy, Rmax, ah, pa, ellip):
    
    rx = cx*np.cos(np.deg2rad(pa)) - cy*np.sin(np.deg2rad(pa))
    ry = cx*np.sin(np.deg2rad(pa)) + cy*np.cos(np.deg2rad(pa))
    
    radius = np.sqrt(rx**2. + (ry**2./(1. - ellip)**2.))
    
    #model1 = 2.*ah**2.*radius*np.sqrt((Rmax**2. + ah**2.)*(Rmax**2. + (1. - ellip)**2.*ah**2.)) / \
    #((1 - ellip) * Rmax**2. * (ah**2. + radius**2.)**2.)
    #model2 = 2.*np.pi*radius/(np.pi*Rmax**2.)
    
    model1 = ah**2.*np.sqrt((Rmax**2. + ah**2.)*(Rmax**2. + (1. - ellip)**2.*ah**2.)) / \
    ((1 - ellip) * Rmax**2. * (ah**2. + radius**2.)**2.)
    model2 = np.pi/(np.pi*Rmax**2.)
    return model1, model2

################################################################
#### Translate the mean proper motions into velocities

def pm2vel(mue, mun, dist):
    
    # Total center of mass (CM) proper motion
    mutran = (mue**2 + mun**2)**(0.5)
    
    # angle of motion for the CM PM and convert from traditional definition of on-sky angles
    thtran = np.deg2rad(np.rad2deg(np.arctan2(mue, mun)) + 90.0)
    
    # total CM velocity
    vtran = 4.7403885 * mutran * dist
    
    return thtran, vtran

################################################################

################################################################
#### Convert to Gaia cartesian coordinates

def wcs2gaiaxy(ra, dec, center):
    x = np.cos(dec) * np.sin(ra-center[0])
    y = np.sin(dec) * np.cos(center[1]) - np.cos(dec) * np.sin(center[1]) * np.cos(ra - center[0])

    x = np.rad2deg(x)
    y = np.rad2deg(y)

    return x,y

################################################################

################################################################
#### Function to convert X/Y coordinates (that were transformed from RA/Dec) to angular coordinates

def wcsxy2ang(x, y, dist0):
    
    rho = np.arctan2( ((x**2 + y**2)**(0.5)), (dist0**(0.5)))
    phi = np.arctan2(y, x)

    return rho, phi

################################################################

##########################################################################
#### Function to convert from WCS to angular coordinates

def wcs2ang(ra0, dec0, ra, dec):

    rho = np.arccos( np.cos(dec) * np.cos(dec0) * np.cos(ra - ra0) + np.sin(dec) * np.sin(dec0) )

    phi = np.arccos( (-1.0*np.cos(dec) * np.sin(ra - ra0)) / (np.sin(rho)) )

# Calculate Cartesian coordinates to test for correcting the angle

    testx, testy = wcs2gaiaxy(ra, dec, np.asarray([ra0, dec0]))

    for i in range(len(testy)):
        if (testy[i] < 0.0):
            phi[i] = -1.0*phi[i]

    return rho, phi

##########################################################################

##########################################################################
#### Function to create vectors (v1, v2, v3) for the center of 
#### mass motion

def make_cm_angvec(vtran, thtran, vsys, rho, phi):

    v1 = vtran*np.sin(rho)*np.cos(phi - thtran) + vsys*np.cos(rho)
    v2 = vtran*np.cos(rho)*np.cos(phi - thtran) - vsys*np.sin(rho)
    v3 = -1.0 * vtran * np.sin(phi - thtran)

    return v1, v2, v3

##########################################################################

##########################################################################
#### Function to calculate the Gamma factor for vector transformations

def calc_gamma(ra0, dec0, ra, dec, rho):

    cosG = (np.sin(dec) * np.cos(dec0) * np.cos(ra-ra0) - np.cos(dec)*np.sin(dec0)) / np.sin(rho)

    sinG = (np.cos(dec0) * np.sin(ra - ra0)) / np.sin(rho)

    return cosG, sinG

##########################################################################

##########################################################################
#### Function to convert an angular vector to wcs

def ang2wcs_vec(dist0, v2, v3, cosG, sinG, rho, phi, theta=np.deg2rad(0.00001), inclin=np.deg2rad(0.00001)):

# Calculates the scaling quantity for the proper motion

    propercon = (np.cos(inclin) * np.cos(rho) - np.sin(inclin)*np.sin(rho)*np.sin(phi-theta)) \
    / (dist0 * np.cos(inclin))

# Use the scaling quantity and the vector components in the skyplane
# to calculate the observable proper motions 

    muwe = (propercon * (-1.0*sinG*(v2) - cosG*(v3))) / (4.7403895)
    muno = (propercon * (cosG*(v2) - sinG*(v3))) / (4.7403895)

    return muwe, muno

##########################################################################

##########################################################################
#### Function to calculate the transverse velocity and angle from given
#### mu_E (mas/yr), mu_N (mas/yr), and distance (kpc)

def calc_transverse(mu_E, mu_N, dist0):

  
    # total CM proper motion
    mu_tran = (mu_E**2 + mu_N**2)**(0.5)
  
    # angle of motion for the CM PM (uses the standard north over east
    # assumption then shifts the angle to refer to quadrant 1)
    thtran = np.deg2rad(np.rad2deg(np.arctan2(mu_E, mu_N)) + 90.0)

    # total CM velocity
    vtran = 4.7403885 * mu_tran * dist0

    return vtran, thtran

################################################################

################################################################
#### Function to calculate the viewing perspective correction
#### for a given system and set of points
#
## Required inputs:
# vtran: the transverse velocity total
# thtran: angle of the transverse vector
# vsys: systemic radial velocity
# dist0: distance to the center of the galaxy
# ra0: RA center of the galaxy
# dec0: Dec center of the galaxy
# ra: column from an array containing the RA coordinates of the points to calculate the CM vector
# dec: column from an array containing the Dec coordinates of the points to calculate the CM vector
# incl: inclination angle of the galaxy to the plane of the sky
# theta: angle of intersection between the galaxy and viewing plane
#
## Outputs:
#

def viewing_cor(muE0, muN0, vsys, dist0, ra0, dec0, ra, dec, inclin=0.0000001, theta=0.0000001):

# Calculate the transverse velocity and angle of the CoM
    vtran, thtran = calc_transverse(muE0, muN0, dist0)

    #print(vtran, np.rad2deg(thtran))
    
# Calculate observational coordinates of the input columns
    rho, phi = wcs2ang(ra0, dec0, ra, dec)

# Calculate the vector components of the CoM motion using given coordinates and parameters
    v1, v2, v3 = make_cm_angvec(vtran, thtran, vsys, rho, phi)


# Calculate the gamma factor for the proper motion calculation
    cosG, sinG = calc_gamma(ra0, dec0, ra, dec, rho)

# Calculate the proper motion for each coordinate
    muw, mun = ang2wcs_vec(dist0, v2, v3, cosG, sinG, rho, phi, theta=theta, inclin=inclin)

# Flip the mu_west PMs to mu_east orientation
    mue = -1.0 * muw

# Subtract the CoM PM from the calculated coordinate PMs to get the residuals
    muecor = mue - muE0
    muncor = mun - muN0

    rvcor = v1 - vsys

    return muecor, muncor, rvcor

##########################################################################

################################################################
#### Function to apply the viewing correction and return new proper motions to be used

def apply_cor(mu_ra1, mu_dec1, dist, vsys, center, data, cx, cy):
    
    # Calculate the direction and scale of the total proper motion vector assumed
    thtran, vtran = pm2vel(mu_ra1, mu_dec1, dist)
        
    # Calculate the viewing perspective correction, assuming a systemic motion of mu_ra1, mu_dec1
    pmracor, pmdeccor, rvcor = viewing_cor(mu_ra1, mu_dec1, vsys, dist, center[0], center[1], \
                                    np.deg2rad(data['ra']), np.deg2rad(data['dec']))
        
    # Remove the corrected systemic motion then re-center the proper motions around the systemic motion
    newpmra = np.copy(data['pmra']) - pmracor
    newpmdec = np.copy(data['pmdec']) - pmdeccor
    
    return newpmra, newpmdec

def pm_gauss_model_wcs(pmra, pmdec, cov, mu_ra, mu_dec):

    diff = np.zeros((len(pmra), 2, 1))
    
    diff[:,0,0] = pmra - mu_ra
    diff[:,1,0] = pmdec - mu_dec
    
    model = np.zeros((len(pmra)))
    
    covdet = cov[:,0,0]*cov[:,1,1] - cov[:,0,1]*cov[:,1,0]
    
    invcov = np.zeros((len(cov), 2, 2))
    
    invcov[:,0,0] = cov[:,0,0] / covdet
    invcov[:,0,1] = cov[:,0,1] / covdet
    invcov[:,1,0] = cov[:,1,0] / covdet
    invcov[:,1,1] = cov[:,1,1] / covdet
    
    model = covdet**(-0.5) * np.exp(-0.5*(diff[:,0,0]*(invcov[:,1,1]*diff[:,0,0]-invcov[:,0,1]*diff[:,1,0]) + \
                                          diff[:,1,0]*(-1.0*invcov[:,1,0]*diff[:,0,0]+invcov[:,0,0]*diff[:,1,0])))
    
    return model


In [8]:
# MW emperical Background MCMC

def total_ln_like(theta, data, cov, cov2, perm, usecor, Rmax, dist, center, cx, cy, vsys,pmMW):
    
    #data, cov, cov2, mu_x1, mu_y1, mu_x2, mu_y2, sigma_x2, sigma_y2 - PM
    # data, Rmax, ah, pa, ellip- spatial
    
    cov_2 = np.copy(cov2) #this line is so that the original covariance matrix doesn't get overwritten
    
       
## permutation for the elliptical spatial case plus proper motion for likelihood
    if (perm=='PM_ellip_spat'):
        frac, mu_ra1, mu_dec1, log_ah, pa, ellip = theta
        #cov_2[:,0,0] += (10**log_sigma_ra2)**2
        #cov_2[:,1,1] += (10**log_sigma_dec2)**2
        
        if (usecor):
            newpmra, newpmdec = apply_cor(mu_ra1, mu_dec1, dist, vsys, center, data, cx, cy)
        else:
            newpmra, newpmdec = data['pmra'], data['pmdec']  
        
        # Calculate the PM likelihoods for each population
        pmmodel1 = pm_gauss_model_wcs(newpmra, newpmdec, cov, mu_ra1, mu_dec1)
        #pmmodel2 = pm_gauss_model_wcs(newpmra, newpmdec, cov_2, mu_ra2, mu_dec2)
        pmmodel2 = pmMW
        # Calculate the spatial likelihoods for each population
        spatmodel1, spatmodel2 = spat_model_ellip(cx, cy, Rmax, 10**log_ah, pa, ellip)
        
        # Combine with the fraction, take the log, then sum over all data points
        like = np.sum(np.log(frac * pmmodel1 * spatmodel1 + (1.0 - frac) * pmmodel2 * spatmodel2))
        
    
    return like


# function to check that all priors are within flat prior range. For now it's set up assuming flat priors. Will
# examine how to modify the setup to include the ability to assign Gaussian priors.

def ln_prior(theta, perm):
    if (perm=='PM_ellip_spat'):
        frac, mu_ra1, mu_dec1, log_ah, pa, ellip = theta
        lp = 0. if (0 < frac < 1) and (-2 <= log_ah < 1) and (-2 < mu_ra1 < 0) and (-2 < mu_dec1 < 0) \
        and (-90. < pa < 90.) and (0. <= ellip < 1.0) else -np.inf
    elif (perm == 'Member_pm_ellip_spat'):   
        mu_ra1, mu_dec1, log_ah, pa, ellip = theta
        lp = 0. if (-2 <= log_ah < 1) and (-2 < mu_ra1 < 0) and (-2 < mu_dec1 < 0) \
        and (-90. < pa < 90.) and (0. <= ellip < 1.0) else -np.inf
    else:
        lp = -np.inf

        
    return lp
        
#def total_ln_like(theta, data, perm='PM_ellip_spat', cov=None, cov2=None, Rmax=None): guesses
def ln_prob(theta, data, cov, cov2, perm, usecor, Rmax, dist, center, cx, cy, vsys,pmMW):
        
    lp = ln_prior(theta, perm)
    if not np.isfinite(lp):
        return -np.inf
    try:
        lnprobval = lp + total_ln_like(theta, data, cov, cov2, perm, usecor, Rmax, dist, center, cx, cy, vsys,pmMW)
    except ValueError: # NaN value case
        lnprobval = -np.inf
        
    return lnprobval

def create_initial(p0, priors):
    
    for a in range(len(p0[0])):
        p0[:,a] = (p0[:,a] * (priors[a][1] - priors[a][0])) + priors[a][[0]]
        
    return p0

def get_priors(perm):
    if perm == 'PM_ellip_spat':
        nparams = 6
        params = ['frac','mu_ra1','mu_dec1','log_ah','pa','ellip']
        priors = np.asarray([[0., 1.], [-2., 0.], [-2., 0.], \
                  [-2, 1], [-90.0, 90.0], [0., 1.0]])
    elif perm == 'Member_pm_ellip_spat':
        nparams = 5
        params = ['mu_ra1','mu_dec1','log_ah','pa','ellip']
        priors = np.asarray([[-2., 0.], [-2., 0.], \
                  [-2, 1], [-90.0, 90.0], [0., 1.0]])
    return priors, nparams, params





In [9]:

def total_ln_like_m(theta, data, cov, cov2, perm, usecor, Rmax, dist, center, cx, cy, vsys):
    
    #data, cov, cov2, mu_x1, mu_y1, mu_x2, mu_y2, sigma_x2, sigma_y2 - PM
    # data, Rmax, ah, pa, ellip- spatial
    cov_2 = np.copy(cov2) #this line is so that the original covariance matrix doesn't get overwritten

## permutation for the elliptical spatial case plus proper motion for likelihood
    if (perm=='Member_pm_ellip_spat'):
        mu_ra1, mu_dec1, log_ah, pa, ellip = theta
        
        if (usecor):
            newpmra, newpmdec = apply_cor(mu_ra1, mu_dec1, dist, vsys, center, data, cx, cy)
        else:
            newpmra, newpmdec = data['pmra'], data['pmdec']  
        
        # Calculate the PM likelihoods for each population
        pmmodel1 = pm_gauss_model_wcs(newpmra, newpmdec, cov, mu_ra1, mu_dec1)
        # Calculate the spatial likelihoods for each population
        spatmodel1, spatmodel2 = spat_model_ellip(cx, cy, Rmax, 10**log_ah, pa, ellip)
        
        # Combine with the fraction, take the log, then sum over all data points
        like = np.sum(np.log(pmmodel1 * spatmodel1))
    return like


# function to check that all priors are within flat prior range. For now it's set up assuming flat priors. Will
# examine how to modify the setup to include the ability to assign Gaussian priors.


#def total_ln_like(theta, data, perm='PM_ellip_spat', cov=None, cov2=None, Rmax=None): guesses
def ln_prob_m(theta, data, cov, cov2, perm, usecor, Rmax, dist, center, cx, cy, vsys):
        
    lp = ln_prior(theta, perm)
    if not np.isfinite(lp):
        return -np.inf
    try:
        lnprobval = lp + total_ln_like_m(theta, data, cov, cov2, perm, usecor, Rmax, dist, center, cx, cy, vsys)
    except ValueError: # NaN value case
        lnprobval = -np.inf
        
    return lnprobval

In [10]:
# 10 parameters MCMC

def total_ln_like_10(theta, data, cov, cov2, perm, usecor, Rmax, dist, center, cx, cy, vsys):
    
    #data, cov, cov2, mu_x1, mu_y1, mu_x2, mu_y2, sigma_x2, sigma_y2 - PM
    # data, Rmax, ah, pa, ellip- spatial
    
    cov_2 = np.copy(cov2) #this line is so that the original covariance matrix doesn't get overwritten
    
    
## permutation for only the proper motion for likelihood
    if (perm == 'PM_only'):
        frac, mu_ra1, mu_dec1, mu_ra2, mu_dec2, sigma_ra2, sigma_dec2 = theta
        cov_2[:,0,0] += sigma_ra2**2
        cov_2[:,1,1] += sigma_dec2**2
        
        if (usecor):
            newpmra, newpmdec = apply_cor(mu_ra1, mu_dec1, dist, vsys, center, data, cx, cy)
        else:
            newpmra, newpmdec = data['pmra'], data['pmdec']
            
        # Calculate the PM likelihoods for each population
        pmmodel1 = pm_gauss_model_wcs(newpmra, newpmdec, cov, mu_ra1, mu_dec1)
        pmmodel2 = pm_gauss_model_wcs(newpmra, newpmdec, cov_2, mu_ra2, mu_dec2)
        
        like = np.sum(np.log(frac * pmmodel1 + (1.0 - frac) * pmmodel2))
        
        
## permutation for only the spherical spatial case
    elif (perm=='sph_spat_only'):
        frac, ah = theta
        spatmodel1, spatmodel2 = spat_model(cx, cy, Rmax, ah)
        like = np.sum(np.log(frac * spatmodel1 + (1.0 - frac) * spatmodel2))
        
        
## permutation for the elliptical spatial case plus proper motion for likelihood
    elif (perm=='ellip_spat_only'):
        frac, ah, pa, ellip = theta
        spatmodel1, spatmodel2 = spat_model_ellip(cx, cy, Rmax, ah, pa, ellip)
        like = np.sum(np.log(frac * spatmodel1 + (1.0 - frac) * spatmodel2))
         
            
## permutation for the spherical spatial case plus proper motion for likelihood
    elif (perm=='PM_sph_spat'):
        frac, mu_x1, mu_y1, mu_x2, mu_y2, sigma_x2, sigma_y2, ah = theta
        cov_2[:,0,0] += sigma_x2**2
        cov_2[:,1,1] += sigma_y2**2

        if (usecor):
            newpmra, newpmdec = apply_cor(mu_ra1, mu_dec1, dist, vsys, center, data, cx, cy)
        else:
            newpmra, newpmdec = data['pmra'], data['pmdec']        

        # Calculate the PM likelihoods for each population
        pmmodel1 = pm_gauss_model_wcs(newpmra, newpmdec, cov, mu_ra1, mu_dec1)
        pmmodel2 = pm_gauss_model_wcs(newpmra, newpmdec, cov_2, mu_ra2, mu_dec2)
        
        spatmodel1, spatmodel2 = spat_model(cx, cy, Rmax, ah)
        like = np.sum(np.log(frac * pmmodel1 * spatmodel1 + (1.0 - frac) * pmmodel2 * spatmodel2))
        
## permutation for the elliptical spatial case plus proper motion for likelihood
    elif (perm=='PM_ellip_spat'):
        frac, mu_ra1, mu_dec1, mu_ra2, mu_dec2, log_sigma_ra2, log_sigma_dec2, log_ah, pa, ellip = theta
        cov_2[:,0,0] += (10**log_sigma_ra2)**2
        cov_2[:,1,1] += (10**log_sigma_dec2)**2
        
        if (usecor):
            newpmra, newpmdec = apply_cor(mu_ra1, mu_dec1, dist, vsys, center, data, cx, cy)
        else:
            newpmra, newpmdec = data['pmra'], data['pmdec']  
        
        # Calculate the PM likelihoods for each population
        pmmodel1 = pm_gauss_model_wcs(newpmra, newpmdec, cov, mu_ra1, mu_dec1)
        pmmodel2 = pm_gauss_model_wcs(newpmra, newpmdec, cov_2, mu_ra2, mu_dec2)
        
        # Calculate the spatial likelihoods for each population
        spatmodel1, spatmodel2 = spat_model_ellip(cx, cy, Rmax, 10**log_ah, pa, ellip)
        
        # Combine with the fraction, take the log, then sum over all data points
        like = np.nansum(np.log(frac * pmmodel1 * spatmodel1 + (1.0 - frac) * pmmodel2 * spatmodel2))
        
    
    return like


# function to check that all priors are within flat prior range. For now it's set up assuming flat priors. Will
# examine how to modify the setup to include the ability to assign Gaussian priors.

def ln_prior_10(theta, perm):
    if (perm == 'PM_only'):
        frac, mu_ra1, mu_dec1, mu_ra2, mu_dec2, sigma_ra2, sigma_dec2 = theta
        
        lp = 0. if (0 < frac < 1) and (0.01 <= sigma_ra2 <= 10) and (0.01 <= sigma_dec2 <= 10) \
        and (-1 < mu_ra1 < 1) and (-1 < mu_dec1 < 1) and (-5 < mu_ra2 < 5) and (-5 < mu_dec2 < 5) else -np.inf
        
        
    elif (perm == 'PM_wcs_only'):
        frac, mu_ra1, mu_dec1, mu_ra2, mu_dec2, sigma_ra2, sigma_dec2 = theta
        
        lp = 0. if (0 < frac < 1) and (0.01 <= sigma_ra2 <= 10) and (0.01 <= sigma_dec2 <= 10) \
        and (-1 < mu_ra1 < 1) and (-1 < mu_dec1 < 1) and (-5 < mu_ra2 < 5) and (-5 < mu_dec2 < 5) else -np.inf
        
        
    elif (perm == 'sph_spat_only'):
        frac, ah = theta
        
        lp = 0. if (0 < frac < 1) and (0.01 <= ah < 1) else -np.inf
        
    elif (perm=='ellip_spat_only'):
        frac, ah, pa, ellip = theta
        
        lp = 0. if (0 < frac < 1) and (0.01 <= ah < 1) and (-90.0 < pa < 90.) and (0. <= ellip < 0.8) else -np.inf
        
        
    elif (perm=='PM_sph_spat'):
        frac, mu_ra1, mu_dec1, mu_ra2, mu_dec2, sigma_ra2, sigma_dec2, ah = theta
        
        lp = 0. if (0<frac<1) and (0.01 <= ah < 1) and (0.01 <= sigma_ra2 <= 10) and (0.01 <= sigma_dec2 <= 10) \
        and (-1 < mu_ra1 < 1) and (-1 < mu_dec1 < 1) and (-5 < mu_ra2 < 5) and (-5 < mu_dec2 < 5) else -np.inf
        
    elif (perm=='PM_ellip_spat'):
        frac, mu_ra1, mu_dec1, mu_ra2, mu_dec2, log_sigma_ra2, log_sigma_dec2, log_ah, pa, ellip = theta
        
        lp = 0. if (0 < frac < 1) and (-2 <= log_ah < 0) and (-2 <= log_sigma_ra2 <= 1) and \
        (-2 <= log_sigma_dec2 <= 1) and (-1 < mu_ra1 < 1) and (-1 < mu_dec1 < 1) and (-5 < mu_ra2 < 5) and \
        (-5 < mu_dec2 < 5) and (-90. < pa < 90.) and (0. <= ellip < 1.0) else -np.inf
        
    else:
        lp = -np.inf

        
    return lp
        
#def total_ln_like(theta, data, perm='PM_ellip_spat', cov=None, cov2=None, Rmax=None): guesses
def ln_prob_10(theta, data, cov, cov2, perm, usecor, Rmax, dist, center, cx, cy, vsys):
        
    lp = ln_prior_10(theta, perm)
    if not np.isfinite(lp):
        return -np.inf
    try:
        lnprobval = lp + total_ln_like_10(theta, data, cov, cov2, perm, usecor, Rmax, dist, center, cx, cy, vsys)
    except ValueError: # NaN value case
        lnprobval = -np.inf
        
    return lnprobval


def get_priors_10(perm):
    
   
    if perm == 'PM_ellip_spat':
        nparams = 10
        params = ['frac','mu_ra1','mu_dec1','mu_ra2','mu_dec2', 'log_sigma_ra2','log_sigma_dec2','log_ah','pa','ellip']
        priors = np.asarray([[0., 1.], [-1., 1.], [-1., 1.], [-5., 5.], [-5., 5.], [-2, 1], [-2, 1.], \
                  [-2, 0], [-90.0, 90.0], [0., 1.0]])
        
    return priors, nparams, params


In [11]:
#pmembership probability for MW background
def posteriors(data,dist,pmMW,cov, cov_2, results, perm,usecor=True):
    if (perm=='PM_only'):
        frac = np.median(results[:,0])
        mu_ra1 = np.median(results[:,1])
        mu_dec1 = np.median(results[:,2])
        #cov_2[:,0,0] += sigma_ra2**2
        #cov_2[:,1,1] += sigma_dec2**2
        if (usecor):
            newpmra, newpmdec = apply_cor(mu_ra1, mu_dec1, dist, vsys, center, data, cx, cy)
        else:
            newpmra, newpmdec = data['pmra'], data['pmdec']  
        # Calculate the PM likelihoods for each population
        pmmodel1 = pm_gauss_model_wcs(newpmra, newpmdec, cov, mu_ra1, mu_dec1)
        #pmmodel2 = pm_gauss_model_wcs(newpmra, newpmdec, cov_2, mu_ra2, mu_dec2)
        pmmodel2 = pmMW
        like = np.sum(np.log(frac * pmmodel1 + (1.0 - frac) * pmmodel2))
        Tot = frac * pmmodel1  + (1.0 - frac) * pmmodel2 
        model1 = frac * pmmodel1 
        P = model1/Tot
        return P
    
    if (perm=='PM_ellip_spat'):
        frac = np.median(results[:,0])
        mu_ra1 = np.median(results[:,1])
        mu_dec1 = np.median(results[:,2])
        #mu_ra2 = np.median(results[:,3])
        #mu_dec2 = np.median(results[:,4])
        #sigma_ra2 = 10**np.median(results[:,5])
        #sigma_dec2 = 10**np.median(results[:,6])
        ah = 10**np.median(results[:,3])
        pa = np.median(results[:,4])
        ellip = np.median(results[:,5])
        #print(sigma_ra2,sigma_dec2,ah)
        #cov_2[:,0,0] += sigma_ra2**2
        #cov_2[:,1,1] += sigma_dec2**2
        if (usecor):
            newpmra, newpmdec = apply_cor(mu_ra1, mu_dec1, dist, vsys, center, data, cx, cy)
        else:
            newpmra, newpmdec = data['pmra'], data['pmdec'] 
        # Calculate the PM likelihoods for each population
        pmmodel1 = pm_gauss_model_wcs(newpmra, newpmdec, cov, mu_ra1, mu_dec1)
        #pmmodel2 = pm_gauss_model_wcs(newpmra, newpmdec, cov_2, mu_ra2, mu_dec2)
        pmmodel2 = pmMW
        spatmodel1, spatmodel2 = spat_model_ellip(cx, cy, Rmax, ah, pa, ellip)
        # Combine with the fraction, take the log, then sum over all data points
        like = np.sum(np.log(frac * pmmodel1 * spatmodel1 + (1.0 - frac) * pmmodel2 * spatmodel2))
        Tot = frac * pmmodel1 * spatmodel1 + (1.0 - frac) * pmmodel2 * spatmodel2
        model1 = frac * pmmodel1 * spatmodel1
        P = model1/Tot
        return P
        

In [12]:
# membership probability for 10 parameters
def posteriors_10(data, dist, cov, cov_2, results, perm, usecor=True):
    if (perm=='PM_only'):
        frac = np.median(results[:,0])
        mu_ra1 = np.median(results[:,1])
        mu_dec1 = np.median(results[:,2])
        mu_ra2 = np.median(results[:,3])
        mu_dec2 = np.median(results[:,4])
        sigma_ra2 = 10**np.median(results[:,5])
        sigma_dec2 = 10**np.median(results[:,6])
        cov_2[:,0,0] += sigma_ra2**2
        cov_2[:,1,1] += sigma_dec2**2
        if (usecor):
            newpmra, newpmdec = apply_cor(mu_ra1, mu_dec1, dist, vsys, center, data, cx, cy)
        else:
            newpmra, newpmdec = data['pmra'], data['pmdec'] 
        # Calculate the PM likelihoods for each population
        pmmodel1 = pm_gauss_model_wcs(newpmra, newpmdec, cov, mu_ra1, mu_dec1)
        pmmodel2 = pm_gauss_model_wcs(newpmra, newpmdec, cov_2, mu_ra2, mu_dec2)
        like = np.sum(np.log(frac * pmmodel1 + (1.0 - frac) * pmmodel2))
        Tot = frac * pmmodel1  + (1.0 - frac) * pmmodel2 
        model1 = frac * pmmodel1 
        P = model1/Tot
        return P
    
    if (perm=='PM_ellip_spat'):
        frac = np.median(results[:,0])
        mu_ra1 = np.median(results[:,1])
        mu_dec1 = np.median(results[:,2])
        mu_ra2 = np.median(results[:,3])
        mu_dec2 = np.median(results[:,4])
        sigma_ra2 = 10**np.median(results[:,5])
        sigma_dec2 = 10**np.median(results[:,6])
        ah = 10**np.median(results[:,7])
        pa = np.median(results[:,8])
        ellip = np.median(results[:,9])
        cov_2[:,0,0] += sigma_ra2**2
        cov_2[:,1,1] += sigma_dec2**2
        if (usecor):
            newpmra, newpmdec = apply_cor(mu_ra1, mu_dec1, dist, vsys, center, data, cx, cy)
        else:
            newpmra, newpmdec = data['pmra'], data['pmdec']
        # Calculate the PM likelihoods for each population
        pmmodel1 = pm_gauss_model_wcs(newpmra, newpmdec, cov, mu_ra1, mu_dec1)
        pmmodel2 = pm_gauss_model_wcs(newpmra, newpmdec, cov_2, mu_ra2, mu_dec2)
        spatmodel1, spatmodel2 = spat_model_ellip(cx, cy, Rmax, ah, pa, ellip)
        # Combine with the fraction, take the log, then sum over all data points
        like = np.sum(np.log(frac * pmmodel1 * spatmodel1 + (1.0 - frac) * pmmodel2 * spatmodel2))
        Tot = frac * pmmodel1 * spatmodel1 + (1.0 - frac) * pmmodel2 * spatmodel2
        model1 = frac * pmmodel1 * spatmodel1
        P = model1/Tot
        return P
       